# Part 2: Time Series Modeling

In this notebook, you will implement functions to extract features from time series data and build ARIMA models.

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from pathlib import Path
import os

# Set style for plots
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Feature Extraction

Implement the `extract_time_series_features` function to calculate rolling window features.

In [2]:
from statsmodels.tsa.stattools import acf

def extract_time_series_features(data, window_size=60):
    """Extract rolling window features from time series data.
    
    Parameters
    ----------
    data : pd.DataFrame
        Preprocessed physiological data
    window_size : int
        Size of the rolling window in seconds
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing extracted features for each signal
    """
    # Initialize a dictionary to store features
    features = {}
    
    # Process each signal column
    for column in data.columns:
        # Skip any non-numeric columns
        if not pd.api.types.is_numeric_dtype(data[column]):
            continue
            
        signal = data[column]
        
        # Basic rolling statistics
        rolling_mean = signal.rolling(f'{window_size}s').mean()
        rolling_std = signal.rolling(f'{window_size}s').std()
        rolling_min = signal.rolling(f'{window_size}s').min()
        rolling_max = signal.rolling(f'{window_size}s').max()
        
        # Calculate range
        rolling_range = rolling_max - rolling_min
        
        # Calculate median and quantiles
        rolling_median = signal.rolling(f'{window_size}s').median()
        rolling_q25 = signal.rolling(f'{window_size}s').quantile(0.25)
        rolling_q75 = signal.rolling(f'{window_size}s').quantile(0.75)
        
        # Calculate IQR
        rolling_iqr = rolling_q75 - rolling_q25
        
        # Calculate skewness and kurtosis
        rolling_skew = signal.rolling(f'{window_size}s').apply(lambda x: pd.Series(x).skew())
        rolling_kurt = signal.rolling(f'{window_size}s').apply(lambda x: pd.Series(x).kurt())
        
        # Function to compute lag-1 autocorrelation for a window
        def compute_autocorr(x):
            if len(x) < 2 or np.all(x == x[0]):
                return np.nan
            try:
                acf_result = acf(x, nlags=1, fft=False)
                return acf_result[1]  # lag-1 autocorrelation
            except:
                return np.nan
        
        # Calculate lag-1 autocorrelation
        rolling_autocorr = signal.rolling(f'{window_size}s').apply(compute_autocorr)
        
        # Store features with column name prefix
        features[f'{column}_mean'] = rolling_mean
        features[f'{column}_std'] = rolling_std
        features[f'{column}_min'] = rolling_min
        features[f'{column}_max'] = rolling_max
        features[f'{column}_range'] = rolling_range
        features[f'{column}_median'] = rolling_median
        features[f'{column}_q25'] = rolling_q25
        features[f'{column}_q75'] = rolling_q75
        features[f'{column}_iqr'] = rolling_iqr
        features[f'{column}_skew'] = rolling_skew
        features[f'{column}_kurt'] = rolling_kurt
        features[f'{column}_autocorr'] = rolling_autocorr
    
    # Combine all features into a single DataFrame
    features_df = pd.DataFrame(features, index=data.index)
    
    # Drop NaN values that occur at the beginning due to the window size
    features_df = features_df.dropna()
    
    return features_df

## 2. ARIMA Modeling

Implement the `build_arima_model` function to fit ARIMA models and generate diagnostic plots.

In [3]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def build_arima_model(series, order=(1,1,1), output_dir='plots'):
    """Fit an ARIMA model to the time series and generate diagnostic plots.
    
    Parameters
    ----------
    series : pd.Series
        Time series data to model
    order : tuple
        (p,d,q) order of the ARIMA model
    output_dir : str
        Directory to save diagnostic plots
        
    Returns
    -------
    statsmodels.tsa.arima.model.ARIMAResults
        Fitted ARIMA model
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Fit ARIMA model
    model = ARIMA(series, order=order)
    model_fit = model.fit()
    
    # 2. Generate diagnostic plots
    
    # Model fit plot - actual vs fitted values
    plt.figure(figsize=(12, 6))
    plt.plot(series, label='Original Data')
    plt.plot(model_fit.fittedvalues, color='red', label='Fitted Values')
    plt.title(f'ARIMA{order} - Original vs Fitted Values')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{output_dir}/model_fit.png')
    plt.close()
    
    # Residuals plot
    residuals = pd.DataFrame(model_fit.resid)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 8))
    
    # Residuals time plot
    residuals.plot(ax=axes[0, 0])
    axes[0, 0].set_title('Residuals')
    
    # Residuals histogram
    residuals.plot(kind='hist', density=True, ax=axes[0, 1])
    axes[0, 1].set_title('Residuals Distribution')
    
    # Residuals ACF
    plot_acf(residuals, ax=axes[1, 0])
    axes[1, 0].set_title('Residuals ACF')
    
    # Residuals PACF
    plot_pacf(residuals, ax=axes[1, 1])
    axes[1, 1].set_title('Residuals PACF')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/residuals_diagnostics.png')
    plt.close()
    
    # 3. Forecast plot
    # Generate forecasts for the next 10 periods
    forecast_steps = 10
    forecast = model_fit.get_forecast(steps=forecast_steps)
    forecast_values = forecast.predicted_mean
    conf_int = forecast.conf_int()
    
    # Create a forecast plot
    plt.figure(figsize=(12, 6))
    plt.plot(series, label='Original Data')
    
    # Create a date range for the forecast
    if isinstance(series.index, pd.DatetimeIndex):
        # If the index is a DatetimeIndex, extend it properly
        forecast_index = pd.date_range(start=series.index[-1], periods=forecast_steps+1, freq=series.index.freq)[1:]
    else:
        # If the index is numeric, just extend it
        forecast_index = np.arange(len(series), len(series) + forecast_steps)
    
    # Plot the forecast
    plt.plot(forecast_index, forecast_values, color='red', label='Forecast')
    plt.fill_between(forecast_index, 
                    conf_int.iloc[:, 0], 
                    conf_int.iloc[:, 1], 
                    color='pink', alpha=0.3, label='95% Confidence Interval')
    
    plt.title(f'ARIMA{order} Forecast')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f'{output_dir}/forecast.png')
    plt.close()
    
    # Return the fitted model
    return model_fit